In [ ]:
# default_exp __init__

In [ ]:
# hide
import os
notebooks_dir = os.getcwd()
project_dir = os.path.dirname(notebooks_dir)

import sys
sys.path.append(project_dir)

# Bitbank

In [ ]:
from ccstabilizer import Exchange
from ccstabilizer import secrets

In [ ]:
# export
import contextlib, requests
from decimal import Decimal

import python_bitbankcc


class Bitbank(Exchange):

    def __init__(self):
        super().__init__()
        self.pub = python_bitbankcc.public()
        self.prv = python_bitbankcc.private(
            os.environ['BITBANK_API_KEY'],
            os.environ['BITBANK_API_SECRET']
        )
        self.update_trading_specifications()

    def update_trading_specification(self, crypto_symbol, fiat_symbol):
        return self.update_trading_specifications().get((crypto_symbol, fiat_symbol), {})

    def update_trading_specifications(self):
        with contextlib.closing(requests.get('https://api.bitbank.cc/v1/spot/status')) as response:
            specs = response.json().get('data', {}).get('statuses', [])
        for spec in specs:
            crypto_symbol, fiat_symbol = spec.get('pair', '_').split('_')
            crypto_symbol, fiat_symbol = crypto_symbol.upper(), fiat_symbol.upper()
            trading_spec = self.trading_specifications.setdefault((crypto_symbol, fiat_symbol), {})
            if 'min_amount' in spec:
                trading_spec['min_trade_unit'] = Decimal(spec.get('min_amount', '0.0001'))
        with contextlib.closing(requests.get('https://api.bitbank.cc/v1/spot/pairs')) as response:
            specs = response.json().get('data', {}).get('pairs', [])
        for spec in specs:
            crypto_symbol, fiat_symbol = spec.get('name', '_').split('_')
            crypto_symbol, fiat_symbol = crypto_symbol.upper(), fiat_symbol.upper()
            trading_spec = self.trading_specifications.setdefault((crypto_symbol, fiat_symbol), {})
            if 'taker_fee_rate_quote' in spec:
                trading_spec['fee_rate'] = Decimal(spec.get('taker_fee_rate_quote', '0.0012'))
            if 'is_enabled' in spec:
                trading_spec['liquid'] = spec.get('is_enabled', False)
        return self.trading_specifications

    def get_portfolio(self):
        return {
            asset.get('asset', '').upper(): Decimal(asset.get('free_amount', '0')) for asset in self.prv.get_asset().get('assets', [])
        }

    def get_price(self, crypto_symbol, fiat_symbol):
        now_ticker = self.pub.get_ticker(f'{crypto_symbol.lower()}_{fiat_symbol.lower()}')
        now_buy_fiat_price_without_fee = Decimal(now_ticker.get('sell', 'Infinity'))
        now_sell_fiat_price_without_fee = Decimal(now_ticker.get('buy', '0'))
        fee_rate = self.get_trading_specification(crypto_symbol, fiat_symbol).get('fee_rate', 0)
        return {
            'now_buy_fiat_price': self.get_buy_fiat_price(now_buy_fiat_price_without_fee, fee_rate),
            'now_sell_fiat_price': self.get_sell_fiat_price(now_sell_fiat_price_without_fee, fee_rate),
            'now_buy_fiat_price_without_fee': now_buy_fiat_price_without_fee,
            'now_sell_fiat_price_without_fee': now_sell_fiat_price_without_fee,
        }
        now_ticker = self.mxc.get_ticker(f'{crypto_symbol}_{fiat_symbol}')[0]
        now_buy_fiat_price_without_fee = Decimal(now_ticker.get('ask', 'Infinity'))
        now_sell_fiat_price_without_fee = Decimal(now_ticker.get('bid', '0'))
        fee_rate = self.get_trading_specification(crypto_symbol, fiat_symbol).get('fee_rate', 0)
        return {
            'now_buy_fiat_price': self.get_buy_fiat_price(now_buy_fiat_price_without_fee, fee_rate),
            'now_sell_fiat_price': self.get_sell_fiat_price(now_sell_fiat_price_without_fee, fee_rate),
            'now_buy_fiat_price_without_fee': now_buy_fiat_price_without_fee,
            'now_sell_fiat_price_without_fee': now_sell_fiat_price_without_fee,
        }

    def get_buy_fiat_price(self, fiat_price_without_fee, fee_rate):
        return fiat_price_without_fee * (1 + fee_rate)

    def get_sell_fiat_price(self, fiat_price_without_fee, fee_rate):
        return fiat_price_without_fee * (1 - fee_rate)

    def buy(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee):
        if amount <= 0:
            raise Exception('Bitbank::buy  amount <= 0')
        self.prv.order(
            pair=f'{crypto_symbol.lower()}_{fiat_symbol.lower()}', price=str(fiat_price_without_fee),
            amount=str(amount), side='buy', order_type='market'
        )

    def sell(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee):
        if amount <= 0:
            raise Exception('Bitbank::sell  amount <= 0')
        self.prv.order(
            pair=f'{crypto_symbol.lower()}_{fiat_symbol.lower()}', price=str(fiat_price_without_fee),
            amount=str(amount), side='sell', order_type='market'
        )

    def has_enough_unused_fiat_money(self, buy_fiat_money, unused_fiat_money):
        return buy_fiat_money <= unused_fiat_money * Decimal('0.75')

    def is_trade_fiat_money_larger_than_limit(self, crypto_symbol, fiat_symbol, amount, fiat_price_without_fee):
        return amount > 0

## Bitbank API
https://github.com/bitbankinc/bitbank-api-docs

In [ ]:
import contextlib, requests
with contextlib.closing(requests.get('https://api.bitbank.cc/v1/spot/status')) as response:
    resp = response.json().get('data', {})
resp

{'statuses': [{'pair': 'btc_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'xrp_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'eth_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'ltc_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'bcc_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'mona_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'xlm_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'qtum_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'bat_jpy', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'xrp_btc', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'eth_btc', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'ltc_btc', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'bcc_btc', 'status': 'NORMAL', 'min_amount': '0.0001'},
  {'pair': 'mona_btc', 'status': 'NORMAL', 'min_amount': '0.0010'},
  {'pair': 'xlm_btc', 'status': 'NORMAL', 'min_

In [ ]:
import contextlib, requests
with contextlib.closing(requests.get('https://api.bitbank.cc/v1/spot/pairs')) as response:
    resp = response.json().get('data', {})
resp

{'pairs': [{'name': 'btc_jpy',
   'base_asset': 'btc',
   'quote_asset': 'jpy',
   'maker_fee_rate_base': '0',
   'taker_fee_rate_base': '0',
   'maker_fee_rate_quote': '0',
   'taker_fee_rate_quote': '0.0005',
   'unit_amount': '0.0001',
   'limit_max_amount': '1000',
   'market_max_amount': '10',
   'market_allowance_rate': '0.2',
   'price_digits': 0,
   'amount_digits': 4,
   'is_enabled': True,
   'stop_order': False,
   'stop_order_and_cancel': False},
  {'name': 'xrp_jpy',
   'base_asset': 'xrp',
   'quote_asset': 'jpy',
   'maker_fee_rate_base': '0',
   'taker_fee_rate_base': '0',
   'maker_fee_rate_quote': '-0.0002',
   'taker_fee_rate_quote': '0.0012',
   'unit_amount': '0.0001',
   'limit_max_amount': '5000000',
   'market_max_amount': '100000',
   'market_allowance_rate': '0.3',
   'price_digits': 3,
   'amount_digits': 4,
   'is_enabled': True,
   'stop_order': False,
   'stop_order_and_cancel': False},
  {'name': 'eth_jpy',
   'base_asset': 'eth',
   'quote_asset': 'jpy',